<center><img src="https://github.com/hse-ds/iad-applied-ds/blob/master/2021/hw/hw1/img/logo_hse.png?raw=1" width="1000"></center>

<h1><center>Прикладные задачи анализа данных</center></h1>
<h2><center>Домашнее задание 4: рекомендательные системы</center></h2>

# Введение

В этом задании Вы продолжите работать с данными из семинара [Articles Sharing and Reading from CI&T Deskdrop](https://www.kaggle.com/gspmoreira/articles-sharing-reading-from-cit-deskdrop).

# Загрузка и предобработка данных

In [1]:
import pandas as pd
import numpy as np
import math

Загрузим данные и проведем предобраотку данных как на семинаре.

In [2]:
!kaggle datasets download -d gspmoreira/articles-sharing-reading-from-cit-deskdrop
#!unzip articles-sharing-reading-from-cit-deskdrop.zip -d articles

articles-sharing-reading-from-cit-deskdrop.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import zipfile
with zipfile.ZipFile('C:/Users/User/articles-sharing-reading-from-cit-deskdrop.zip', 'r') as zip_ref:
    zip_ref.extractall('C:/Users/User')

In [4]:
articles_df = pd.read_csv("shared_articles.csv")
articles_df = articles_df[articles_df["eventType"] == "CONTENT SHARED"]
articles_df.head(2)

,timestamp,eventType,contentId,authorPersonId,authorSessionId,authorUserAgent,authorRegion,authorCountry,contentType,url,title,text,lang
1,1459193988,CONTENT SHARED,-4110354420726924665,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://www.nytimes.com/2016/03/28/business/dea...,"Ethereum, a Virtual Currency, Enables Transact...",All of this work is still very early. The firs...,en
2,1459194146,CONTENT SHARED,-7292285110016212249,4340306774493623681,8940341205206233829,NaN,NaN,NaN,HTML,http://cointelegraph.com/news/bitcoin-future-w...,Bitcoin Future: When GBPcoin of Branson Wins O...,The alarm clock wakes me at 8:00 with stream o...,en


In [5]:
interactions_df = pd.read_csv("users_interactions.csv")
interactions_df.head(2)

,timestamp,eventType,contentId,personId,sessionId,userAgent,userRegion,userCountry
0,1465413032,VIEW,-3499919498720038879,-8845298781299428018,1264196770339959068,NaN,NaN,NaN
1,1465412560,VIEW,8890720798209849691,-1032019229384696495,3621737643587579081,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...,NY,US


In [6]:
interactions_df.personId = interactions_df.personId.astype(str)
interactions_df.contentId = interactions_df.contentId.astype(str)
articles_df.contentId = articles_df.contentId.astype(str)

In [7]:
# зададим словарь определяющий силу взаимодействия
event_type_strength = {
   "VIEW": 1.0,
   "LIKE": 2.0, 
   "BOOKMARK": 2.5, 
   "FOLLOW": 3.0,
   "COMMENT CREATED": 4.0,  
}

interactions_df["eventStrength"] = interactions_df.eventType.apply(lambda x: event_type_strength[x])

Оставляем только тех пользователей, которые произамодействовали более чем с пятью статьями.

In [8]:
users_interactions_count_df = (
    interactions_df
    .groupby(["personId", "contentId"])
    .first()
    .reset_index()
    .groupby("personId").size())
print("# users:", len(users_interactions_count_df))

users_with_enough_interactions_df = \
    users_interactions_count_df[users_interactions_count_df >= 5].reset_index()[["personId"]]
print("# users with at least 5 interactions:",len(users_with_enough_interactions_df))

# users: 1895
# users with at least 5 interactions: 1140


Оставляем только те взаимодействия, которые относятся к отфильтрованным пользователям.

In [9]:
interactions_from_selected_users_df = interactions_df.loc[np.in1d(interactions_df.personId,
            users_with_enough_interactions_df)]

In [10]:
print(f"# interactions before: {interactions_df.shape}")
print(f"# interactions after: {interactions_from_selected_users_df.shape}")

# interactions before: (72312, 9)
# interactions after: (69868, 9)


Объединяем все взаимодействия пользователя по каждой статье и сглажиываем полученный результат, взяв от него логарифм.

In [11]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"]).eventStrength.sum()
    .apply(smooth_user_preference)
    .reset_index().set_index(["personId", "contentId"])
)
interactions_full_df["last_timestamp"] = (
    interactions_from_selected_users_df
    .groupby(["personId", "contentId"])["timestamp"].last()
)
        
interactions_full_df = interactions_full_df.reset_index()
interactions_full_df.head(5)

,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.000000,1470395911
1,-1007001694607905623,-6623581327558800021,1.000000,1487240080
2,-1007001694607905623,-793729620925729327,1.000000,1472834892
3,-1007001694607905623,1469580151036142903,1.000000,1487240062
4,-1007001694607905623,7270966256391553686,1.584963,1485994324


Разобьём выборку на обучение и контроль по времени.

In [12]:
from sklearn.model_selection import train_test_split

split_ts = 1475519530
interactions_train_df = interactions_full_df.loc[interactions_full_df.last_timestamp < split_ts].copy()
interactions_test_df = interactions_full_df.loc[interactions_full_df.last_timestamp >= split_ts].copy()

print(f"# interactions on Train set: {len(interactions_train_df)}")
print(f"# interactions on Test set: {len(interactions_test_df)}")

interactions_train_df.head(5)

# interactions on Train set: 29329
# interactions on Test set: 9777


,personId,contentId,eventStrength,last_timestamp
0,-1007001694607905623,-5065077552540450930,1.0,1470395911
2,-1007001694607905623,-793729620925729327,1.0,1472834892
6,-1032019229384696495,-1006791494035379303,1.0,1469129122
7,-1032019229384696495,-1039912738963181810,1.0,1459376415
8,-1032019229384696495,-1081723567492738167,2.0,1464054093


Для удобства подсчёта качества запишем данные в формате, где строка соответствует пользователю, а столбцы будут истинными метками и предсказаниями в виде списков.

In [13]:
interactions = (
    interactions_train_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
    .reset_index()
    .rename(columns={"contentId": "true_train"})
    .set_index("personId")
)

interactions["true_test"] = (
    interactions_test_df
    .groupby("personId")["contentId"].agg(lambda x: list(x))
)

# заполнение пропусков пустыми списками
interactions.loc[pd.isnull(interactions.true_test), "true_test"] = [
    "" for x in range(len(interactions.loc[pd.isnull(interactions.true_test), "true_test"]))]

interactions.head(5)

,true_train,true_test
personId,,
-1007001694607905623,"[-5065077552540450930, -793729620925729327]","[-6623581327558800021, 1469580151036142903, 72..."
-1032019229384696495,"[-1006791494035379303, -1039912738963181810, -...","[-1415040208471067980, -2555801390963402198, -..."
-108842214936804958,"[-1196068832249300490, -133139342397538859, -1...","[-2780168264183400543, -3060116862184714437, -..."
-1130272294246983140,"[-1150591229250318592, -1196068832249300490, -...","[-1606980109000976010, -1663441888197894674, -..."
-1160159014793528221,"[-133139342397538859, -387651900461462767, 377...",[-3462051751080362224]


# Библиотека LightFM

Для рекомендации Вы будете пользоваться библиотекой [LightFM](https://making.lyst.com/lightfm/docs/home.html), в которой реализованы популярные алгоритмы. Для оценивания качества рекомендации, как и на семинаре, будем пользоваться метрикой *precision@10*.

In [14]:
#conda install -c conda-forge lightfm
#conda update -n base -c defaults conda

from lightfm import LightFM
from lightfm.evaluation import precision_at_k

C:\Python\ANACONDA\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


## Задание 1 (2 балла)

Модели в LightFM работают с разреженными матрицами. Создайте разреженные матрицы `data_train` и `data_test` (размером количество пользователей на количество статей), такие что на пересечении строки пользователя и столбца статьи стоит сила их взаимодействия, если взаимодействие было, и стоит ноль, если взаимодействия не было.

In [15]:
from scipy.sparse import csr_matrix

In [16]:
#https://medium.com/swlh/an-in-depth-introduction-to-sparse-matrix-a5972d7e8c86

all_users = np.unique(np.hstack((interactions_train_df.personId.unique(), interactions_test_df.personId.unique())))
all_articles =  np.unique(np.hstack((interactions_train_df.contentId.unique(), interactions_test_df.contentId.unique())))

# Пустые матрицы одинакового размера
matrix_train = pd.DataFrame(0, columns=all_articles, index=all_users)
matrix_test = matrix_train.copy()

# Заполняю
for i in range(interactions_train_df.shape[0]):
    strength = np.array(interactions_train_df.eventStrength)[i]
    if strength != np.nan:
        matrix_train.loc[np.array(interactions_train_df.personId)[i], np.array(interactions_train_df.contentId)[i]] = strength
    else:
        matrix_train.loc[np.array(interactions_train_df.personId)[i], np.array(interactions_train_df.contentId)[i]] = 0
        
for i in range(interactions_test_df.shape[0]):
    strength = np.array(interactions_test_df.eventStrength)[i]
    if strength != np.nan:
        matrix_test.loc[np.array(interactions_test_df.personId)[i], np.array(interactions_test_df.contentId)[i]] = strength
    else:
        matrix_test.loc[np.array(interactions_test_df.personId)[i], np.array(interactions_test_df.contentId)[i]] = 0
        

In [17]:
data_train = csr_matrix(np.array(matrix_train))
data_test = csr_matrix(np.array(matrix_test))

In [18]:
assert data_train.shape == data_test.shape
assert data_train.shape[0] == all_users.shape[0] and data_train.shape[1] == all_articles.shape[0]
assert data_test.shape[0] == all_users.shape[0] and data_test.shape[1] == all_articles.shape[0]

## Задание 2 (1 балл)

Обучите модель LightFM с `loss="warp"` и посчитайте *precision@10* на тесте.

In [19]:
%%time

lfm = LightFM(loss='warp')
lfm.fit(data_train, epochs=20)

pr_train = precision_at_k(model = lfm, 
                          test_interactions = data_train, 
                          k=10).mean()
pr_test = precision_at_k(model = lfm, 
                         test_interactions = data_test, 
                         train_interactions=data_train, 
                         k=10).mean()

print('Precision на трейне:', pr_train, '\nPrecision на тесте:', pr_test)

Precision на трейне: 0.21708633 
Precision на тесте: 0.0066191447
Wall time: 3.31 s


In [20]:
# Буду сохранять все пресижены
saved_scores = pd.DataFrame(columns = ['Model', 'Precision on train', 'Precision on test'])
saved_scores = saved_scores.append({'Model': 'Baseline: LightFM with loss = warp',
                                    'Precision on train': pr_train, 'Precision on test': pr_test}, ignore_index = True);

## Задание 3 (3 балла)

При вызове метода `fit` LightFM позволяет передавать в `item_features` признаковое описание объектов. Воспользуемся этим. Будем получать признаковое описание из текста статьи в виде [TF-IDF](https://ru.wikipedia.org/wiki/TF-IDF) (можно воспользоваться `TfidfVectorizer` из scikit-learn). Создайте матрицу `feat` размером количесвто статей на размер признакового описание и обучите LightFM с `loss="warp"` и посчитайте precision@10 на тесте.

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [22]:
%%time

vectorizer = TfidfVectorizer()
feat = vectorizer.fit_transform(articles_df.text)

lfm = LightFM(loss='warp')
lfm.fit(data_train, epochs=20, item_features = feat)

pr_train = precision_at_k(model = lfm, 
                          test_interactions = data_train,
                          item_features = feat,
                          k=10).mean()
pr_test = precision_at_k(model = lfm, 
                         test_interactions = data_test,
                         item_features = feat,
                         train_interactions=data_train, 
                         k=10).mean()

print('Precision на трейне:', pr_train, '\nPrecision на тесте:', pr_test)

Precision на трейне: 0.23570144 
Precision на тесте: 0.006720978
Wall time: 6min 19s


In [23]:
saved_scores = saved_scores.append({'Model': 'Add TF-IDF',
                                    'Precision on train': pr_train, 'Precision on test': pr_test}, ignore_index = True);

## Задание 4 (2 балла)

В задании 3 мы использовали сырой текст статей. В этом задании необходимо сначала сделать предобработку текста (привести к нижнему регистру, убрать стоп слова, привести слова к номральной форме и т.д.), после чего обучите модель и оценить качество на тестовых данных.

In [24]:
#https://github.com/stopwords-iso/stopwords-iso
#https://www.geeksforgeeks.org/python-lemmatization-approaches-with-examples/
#https://stackoverflow.com/questions/24647400/what-is-the-best-stemming-method-in-python

#pip install stopwordsiso
import stopwordsiso as stpwrds
import nltk
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk import word_tokenize

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [25]:
langs = list(articles_df.lang.unique())
articles_df.lang.value_counts()

en    2211
pt     829
la       3
ja       2
es       2
Name: lang, dtype: int64

In [26]:
stopwords_all = stpwrds.stopwords(langs)
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

In [27]:
# Токенайзер который делает bare minimum
def my_tokenizer(text):
    tokens = word_tokenize(text.lower())
    tokens = [lemmatizer.lemmatize(token) for token in tokens \
               if token not in stopwords_all and token.isalpha() == True]
    
    return tokens

In [28]:
%%time

vectorizer = TfidfVectorizer(tokenizer = my_tokenizer)
feat = vectorizer.fit_transform(articles_df.text)
lfm = LightFM(loss='warp')
lfm.fit(data_train, epochs=20, item_features = feat)

pr_train = precision_at_k(model = lfm, 
                          test_interactions = data_train,
                          item_features = feat,
                          k=10).mean()
pr_test = precision_at_k(model = lfm, 
                         test_interactions = data_test,
                         item_features = feat,
                         train_interactions=data_train, 
                         k=10).mean()

print('Precision на трейне:', pr_train, '\nPrecision на тесте:', pr_test)

Precision на трейне: 0.24145684 
Precision на тесте: 0.0065173116
Wall time: 3min 42s


In [29]:
saved_scores = saved_scores.append({'Model': 'TF-IDF + custom tokenizer (lemma)',
                                    'Precision on train': pr_train, 'Precision on test': pr_test}, ignore_index = True);

**Улучшилось ли качество предсказания?**

*Нет, не улучшилось (и вообще улучшается / ухудшается рандомно при разных запусках тетрадки), у меня есть подозрение, что WordNetLemmatizer может плохо справляться с текстами не на английском языке, хотя по документации предполагается, что он без проблем берет лемму от любого слова*

In [30]:
# Проверяю токенайзер на статье на португальском языке (таких у нас много -- больше 800)
pt_texts = np.array(articles_df[articles_df.lang == 'pt'].text)
my_tokenizer(pt_texts[1])

['artigos',
 'palestra',
 'artigos',
 'perspectivas',
 'agronegócio',
 'demandam',
 'tecnologias',
 'produtividade',
 'sustentável',
 'agricultura',
 'precisão',
 'alimentar',
 'bilhões',
 'artigos',
 'adoção',
 'agricultura',
 'precisão',
 'brasil',
 'alberto',
 'campos',
 'bernardi',
 'embrapa',
 'pecuária',
 'sudeste',
 'ricardo',
 'inamasu',
 'embrapa',
 'instrumentação',
 'artigos',
 'agricultura',
 'precisão',
 'ferramenta',
 'alcance',
 'alberto',
 'bernardi',
 'pesquisador',
 'embrapa',
 'pecuária',
 'sudeste',
 'formado',
 'agronomia',
 'mestrado',
 'nutrição',
 'plantas',
 'doutorado',
 'nutrição',
 'plantas',
 'fertilidade',
 'adubação',
 'integração',
 'agricultura',
 'precisão',
 'ricardo',
 'inamasu',
 'pesquisador',
 'embrapa',
 'instrumentação',
 'graduação',
 'mestrado',
 'doutorado',
 'engenharia',
 'mecânica',
 'escola',
 'engenharia',
 'carlos',
 'usp',
 'biological',
 'system',
 'engineering',
 'university',
 'nebraska',
 'lincoln',
 'experiência',
 'engenharia',
 

*Выглядит не очень, у слов как будто остались окончания множественного числа или окончания, обозначающие гендер*

In [31]:
# Перепишу токенайзер со стеммингом вместо лемматизации
def my_tokenizer_stem(text):
    tokens = word_tokenize(text.lower())
    
    tokens = [stemmer.stem(token) for token in tokens \
               if token not in stopwords_all and token.isalpha() == True]
    return tokens

In [32]:
%%time

vectorizer = TfidfVectorizer(tokenizer = my_tokenizer_stem)
feat = vectorizer.fit_transform(articles_df.text)
lfm = LightFM(loss='warp')
lfm.fit(data_train, epochs=20, item_features = feat)

pr_train = precision_at_k(model = lfm, 
                          test_interactions = data_train,
                          item_features = feat,
                          k=10).mean()
pr_test = precision_at_k(model = lfm, 
                         test_interactions = data_test,
                         item_features = feat,
                         train_interactions=data_train, 
                         k=10).mean()

print('Precision на трейне:', pr_train, '\nPrecision на тесте:', pr_test)

Precision на трейне: 0.2365108 
Precision на тесте: 0.0066191447
Wall time: 4min 35s


In [33]:
saved_scores = saved_scores.append({'Model': 'TF-IDF + custom tokenizer (stem)',
                                    'Precision on train': pr_train, 'Precision on test': pr_test}, ignore_index = True);

*Почти ничего не изменилось, ну и ладно*

## Задание 5 (2 балла)

Подберите гиперпараметры модели LightFM (`n_components` и др.) для улучшения качества модели.

In [34]:
#https://stackoverflow.com/questions/49896816/how-do-i-optimize-the-hyperparameters-of-lightfm

import itertools

def sample_hyperparameters():
    
    while True:
        yield {
            "no_components": np.random.randint(16, 64),
            "learning_schedule": np.random.choice(["adagrad", "adadelta"]),
            "learning_rate": np.random.exponential(0.05),
            "item_alpha": np.random.exponential(1e-8),
            "user_alpha": np.random.exponential(1e-8),
            "num_epochs": np.random.randint(5, 50),
        }

def random_search(train, test, num_samples=10):

    for hyperparams in itertools.islice(sample_hyperparameters(), num_samples):
        num_epochs = hyperparams.pop("num_epochs")

        model = LightFM(**hyperparams, loss='warp')
        model.fit(train, epochs=num_epochs, item_features = feat)

        #score = auc_score(model, test, train_interactions=train, num_threads=num_threads).mean()
        score = precision_at_k(model = model,
                               test_interactions = test,
                               item_features = feat,
                               train_interactions=train, 
                               k=10).mean()

        hyperparams["num_epochs"] = num_epochs

        yield (score, hyperparams, model)

In [35]:
%%time
(score, hyperparams, model) = max(random_search(data_train, data_test), key=lambda x: x[0])
print("Best score {} at {}".format(score, hyperparams))

Best score 0.007331975270062685 at {'no_components': 29, 'learning_schedule': 'adadelta', 'learning_rate': 0.04058360639561338, 'item_alpha': 5.616316422287524e-08, 'user_alpha': 3.3506621712832566e-08, 'num_epochs': 29}
Wall time: 2h 17min 38s


In [36]:
saved_scores = saved_scores.append({'Model': 'Random search on hyperparam -- best model',
                                    'Precision on train': '', 'Precision on test': score}, ignore_index = True);

In [37]:
# Сохраняю гиперпараметры для следующего задания
best_hp = hyperparams.copy()

## Бонусное задание (3 балла)

Выше мы использовали достаточно простое представление текста статьи в виде TF-IDF. В этом задании Вам нужно представить текст статьи (можно вместе с заголовком) в виде эмбеддинга полученного с помощью рекуррентной сети или трансформера (можно использовать любую предобученную модель, которая Вам нравится). Обучите модель с ипользованием этих эмеддингов и сравните результаты с предыдущими.

In [38]:
#pip install sentence_transformers
from sentence_transformers import SentenceTransformer

In [39]:
#https://medium.com/swlh/transformer-based-sentence-embeddings-cd0935b3b1e0
#https://sbert.net/examples/applications/computing-embeddings/README.html#sentence-embeddings-with-transformers

transformer = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = transformer.encode(np.array(articles_df.text))
embeddings = csr_matrix(embeddings)

assert embeddings.shape[0] == feat.shape[0]
assert type(embeddings) == type(feat)

In [40]:
%%time

n_epochs = best_hp.pop('num_epochs')
lfm = LightFM(**best_hp, loss='warp')
lfm.fit(data_train, epochs = n_epochs, item_features = embeddings)

pr_train = precision_at_k(model = lfm, 
                          test_interactions = data_train,
                          item_features = embeddings,
                          k=10).mean()
pr_test = precision_at_k(model = lfm, 
                         test_interactions = data_test,
                         item_features = embeddings,
                         train_interactions=data_train, 
                         k=10).mean()

print('Precision на трейне:', pr_train, '\nPrecision на тесте:', pr_test)

Precision на трейне: 0.18408275 
Precision на тесте: 0.004378819
Wall time: 48min 52s


In [57]:
saved_scores = saved_scores.append({'Model': 'Best model (prev), add sentence transformer',
                                    'Precision on train': pr_train, 'Precision on test': pr_test}, ignore_index = True);

In [60]:
# Сравнение результатов
saved_scores

,Model,Precision on train,Precision on test
0,Baseline: LightFM with loss = warp,0.217086,0.006619
1,Add TF-IDF,0.235701,0.006721
2,TF-IDF + custom tokenizer (lemma),0.241457,0.006517
3,TF-IDF + custom tokenizer (stem),0.236511,0.006619
4,Random search on hyperparam -- best model,,0.007332
5,"Best model (prev), add sentence transformer",0.184083,0.004379
